# <center>**Neighbourhoods in Toronto**</center>

### <span style="color:blue">**All the parts of this assignment are in this notebook**</style>

<span style="color:red"> **1. Start by creating a new Notebook for this assignment.** </style>

In [ ]:
# install libraries
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c anaconda lxml --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
print('libraries installed')

In [1]:
# Import Libraries
from bs4 import BeautifulSoup
import requests 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans
import folium # map rendering library
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

<span style="color:red"> **2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe** </style>

In [2]:
# URL with the data of Toronto
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url


'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# Requests to fetch the raw content of the HTML page
html_content = requests.get(url).text

In [4]:
# Parce the HTML content
soup = BeautifulSoup(html_content, 'lxml')

In [5]:
# scrape the text from the file
table = soup.find("table", attrs = {"class" : "wikitable"})
table_data = table.tbody.find_all("tr")

In [6]:
# Building table
headings = []
rows = []
table_scrape = []

for th in table_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())
    
    for tr in table.tbody.find_all("tr"):
        t_row = {}
        
        for td,th in zip(tr.find_all("td"), headings):
            t_row[th] = td.text.replace('\n', '').strip()
        table_scrape.append(t_row)

In [81]:
# Cast the table into a data frame
df = pd.DataFrame(table_scrape) 
print(df.shape)
df.tail()

(864, 3)


,Postcode,Borough,Neighbourhood
859,M8Z,Etobicoke,Mimico NW
860,M8Z,Etobicoke,The Queensway West
861,M8Z,Etobicoke,Royal York South West
862,M8Z,Etobicoke,South of Bloor
863,M9Z,Not assigned,Not assigned


In [82]:
# cleaning the data frame
df2 = df.dropna()
print(df2.shape)
df2.reset_index(inplace = True)
df3 = df2[df2.Borough != 'Not assigned']
df3.reset_index(inplace = True)
df3.drop(['level_0', 'index'], axis = 1, inplace = True)
df3.replace(to_replace ="Not assigned", value ="Queen\'s Park", inplace = True) 
df3.head()

(287, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<span style="color:red"> **3. To create the above dataframe:**

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

<span style="color:red"> **4. Submit a link to your Notebook on your Github repository. (10 marks)** </style>

In [83]:
# Grouping
df4 = df3.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
print(df4.shape)
df4.head(20)

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [84]:
df4.shape

(103, 3)

### <span style="color:red">**In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.**</style>

The geocoder library didn't work, so I use the cvs file provided.

In [85]:
# Reading cvs file
coordinates = pd.read_csv('Geospatial_Coordinates.csv') 
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [86]:
# Mergin the two tables in one
d1 = coordinates.set_index('Postal Code')
d2 = df4.set_index('Postcode')
d3 = d2.join(d1)
d3.reset_index(inplace = True)
print(d3.shape)
d3.head(20)

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [87]:
d3.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke', "Queen's Park"], dtype=object)

# Segmentation

<span style="color:purple">**In this assigment I will use the Boroughs : East Toronto, Central Toronto, Downtown Toronto, and West Toronto**</style>

In [88]:
# Getting the coordinates of Toronto
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### __Map of Toronto with Postcodes and Boroughs superimposed on top__

In [100]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, postcode, borough in zip(d3['Latitude'], d3['Longitude'], d3['Postcode'], d3['Borough']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### __Create a map of Toronto with Boroughs and Neighborhoods superimposed on top__

In [104]:
# Building the Dataframe with the Boroughs needed
temp1 = d3[d3.Borough == 'East Toronto']
temp2 = d3[d3.Borough == 'Central Toronto']
temp3 = d3[d3.Borough == 'Downtown Toronto']
temp4 = d3[d3.Borough == 'West Toronto']
neighborhoods = pd.concat([temp1, temp2, temp3, temp4], ignore_index=True)
print(neighborhoods.shape)
print(neighborhoods.Borough.unique())
neighborhoods.head(10)

(39, 5)
['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
5,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197
7,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
8,M4S,Central Toronto,Davisville,43.704324,-79.388790
9,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [99]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(temp1['Latitude'], temp1['Longitude'], temp1['Borough'], temp1['Neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

for lat, lng, borough, neighbourhood in zip(temp2['Latitude'], temp2['Longitude'], temp2['Borough'], temp2['Neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, borough, neighbourhood in zip(temp3['Latitude'], temp3['Longitude'], temp3['Borough'], temp3['Neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

for lat, lng, borough, neighbourhood in zip(temp4['Latitude'], temp4['Longitude'], temp4['Borough'], temp4['Neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)      

map_toronto

### Define Foursquare Credentials and Version

In [101]:
CLIENT_ID = 'LRHP1G0RTWGHAYIBQANN2PEUTL1JDFOH0L0UVQ2NHQLT20SK' # your Foursquare ID
CLIENT_SECRET = '35XM4U35SLK1UGAOEFW2LV2Z05C2PINU02T1BHVCRN5BAITQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LRHP1G0RTWGHAYIBQANN2PEUTL1JDFOH0L0UVQ2NHQLT20SK
CLIENT_SECRET:35XM4U35SLK1UGAOEFW2LV2Z05C2PINU02T1BHVCRN5BAITQ


##  Explore East Toronto, Central Toronto, Downtown Toronto, and West Toronto neighborhoods

In [118]:
# Funcion to obtain the venues of each neighborhood
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [119]:
neighborhood_toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, 

In [120]:
print(neighborhood_toronto_venues.shape)
neighborhood_toronto_venues.head()

(3202, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


Let's check how many venues were returned for each neighborhood

In [121]:
neighborhood_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,48,48,48,48,48,48
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",37,37,37,37,37,37
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [122]:
print('There are {} uniques categories.'.format(len(neighborhood_toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


##  Analyze Each Neighborhood

In [125]:
# one hot encoding
neighborhood_toronto_onehot = pd.get_dummies(neighborhood_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_toronto_onehot['Neighborhood'] = neighborhood_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_toronto_onehot.columns[-1]] + list(neighborhood_toronto_onehot.columns[:-1])
neighborhood_toronto_onehot = neighborhood_toronto_onehot[fixed_columns]

print(neighborhood_toronto_onehot.shape)
neighborhood_toronto_onehot.head()

(3202, 276)


,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [126]:
neighborhood_toronto_grouped = neighborhood_toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(neighborhood_toronto_grouped.shape)
neighborhood_toronto_grouped.head(10)

(39, 276)


,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.00,0.00,0.0000,0.0000,0.020000

Create the a dataframe and display the top 10 venues for each neighborhood.

In [127]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [186]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_toronto_grouped['Neighborhood']

for ind in np.arange(neighborhood_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head(20)

(39, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Hotel,Coffee Shop,Theater,Restaurant,Sushi Restaurant,Steakhouse,Concert Hall,Bakery,Bar
1,Berczy Park,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Bakery,Cocktail Bar,Park,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Restaurant,Coffee Shop,Bar,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Vegetarian / Vegan Restaurant,Juice Bar
3,Business Reply Mail Processing Centre 969 Eastern,Park,Coffee Shop,Brewery,Pizza Place,Pet Store,Italian Restaurant,Sushi Restaurant,Burrito Place,Restaurant,French Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Café,Harbor / Marina,Dog Run,Sculpture Garden,Park,Track,Dance Studio,Garden,Scenic Lookout
5,"Cabbagetown, St. James Town",Café,Restaurant,Coffee Shop,Gastropub,Japanese Restaurant,Park,Diner,Rock Club,Caribbean Restaurant,Jewelry Store
6,Central Bay Street,Coffee Shop,Japanese Restaurant,Ice Cream Shop,Thai Restaurant,Italian Restaurant,Art Gallery,Bakery,Plaza,Pizza Place,Tea Room
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dessert Shop,Chinese Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Art Gallery
8,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Mexican Restaurant,Cocktail Bar,Ethiopian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Park,Café,Men's Store,Gay Bar,Gym,Burger Joint,Sushi Restaurant,Dance Studio


##  <span style="color:red">**Clustering Neighborhoods**</style>

 *k*-means to cluster the neighborhood into 5 clusters.

In [185]:
# set number of clusters
kclusters = 3

neighborhood_toronto_grouped_clustering = neighborhood_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 2, 0, 2, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [187]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_toronto_merged = neighborhoods

# merge toronto_grouped with neighborhoods to add latitude/longitude for each neighborhood
neighborhood_toronto_merged = neighborhood_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

neighborhood_toronto_merged.head(10) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Coffee Shop,Pizza Place,Beach,Breakfast Spot,Japanese Restaurant,Bar,Caribbean Restaurant,Bakery,Tea Room
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Café,Pub,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Bookstore,Spa,Ramen Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Indian Restaurant,Coffee Shop,Beach,Café,Grocery Store,Bakery,Brewery,Fast Food Restaurant,Harbor / Marina,Park
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bar,Café,American Restaurant,Brewery,Bakery,Italian Restaurant,Vietnamese Restaurant,Diner,French Restaurant
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Park,Coffee Shop,Brewery,Pizza Place,Pet Store,Italian Restaurant,Sushi Restaurant,Burrito Place,Restaurant,French Restaurant
5,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Café,Coffee Shop,Pool,Bookstore,Gym / Fitness Center,College Quad,Park,College Gym,Trail,Ethiopian Restaurant
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Coffee Shop,Italian Restaurant,Café,Pizza Place,Fast Food Restaurant,Dessert Shop,Gym,Sushi Restaurant,Yoga Studio,Diner
7,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Italian Restaurant,Park,Diner,Sporting Goods Shop,Skating Rink,Mexican Restaurant,Café,Rental Car Location,Restaurant
8,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Pizza Place,Indian Restaurant,Middle Eastern Restaurant,Café,Dessert Shop,Gym
9,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Grocery Store,Sandwich Place,Thai Restaurant,Park,Café,Restaurant


**Visualize the resulting clusters**

In [188]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_toronto_merged['Latitude'], neighborhood_toronto_merged['Longitude'], neighborhood_toronto_merged['Neighbourhood'], neighborhood_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## <span style="color:red">**Examine Clusters**</style>

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

### Cluster 1 - Restaurants and bars

In [189]:
c1 = neighborhood_toronto_merged.loc[neighborhood_toronto_merged['Cluster Labels'] == 0].drop(['Latitude','Longitude'], axis=1)
print('The most common venues are {}'.format(c1['1st Most Common Venue'].unique()))
c1.head(15)

The most common venues are ['Pub' 'Greek Restaurant' 'Indian Restaurant' 'Coffee Shop' 'Park'
 'Sushi Restaurant' 'Gym / Fitness Center' 'Café' 'Korean Restaurant']


,Postcode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,0,Pub,Coffee Shop,Pizza Place,Beach,Breakfast Spot,Japanese Restaurant,Bar,Caribbean Restaurant,Bakery,Tea Room
1,M4K,East Toronto,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Café,Pub,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Bookstore,Spa,Ramen Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",0,Indian Restaurant,Coffee Shop,Beach,Café,Grocery Store,Bakery,Brewery,Fast Food Restaurant,Harbor / Marina,Park
3,M4M,East Toronto,Studio District,0,Coffee Shop,Bar,Café,American Restaurant,Brewery,Bakery,Italian Restaurant,Vietnamese Restaurant,Diner,French Restaurant
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0,Park,Coffee Shop,Brewery,Pizza Place,Pet Store,Italian Restaurant,Sushi Restaurant,Burrito Place,Restaurant,French Restaurant
6,M4P,Central Toronto,Davisville North,0,Coffee Shop,Italian Restaurant,Café,Pizza Place,Fast Food Restaurant,Dessert Shop,Gym,Sushi Restaurant,Yoga Studio,Diner
7,M4R,Central Toronto,North Toronto West,0,Coffee Shop,Italian Restaurant,Park,Diner,Sporting Goods Shop,Skating Rink,Mexican Restaurant,Café,Rental Car Location,Restaurant
8,M4S,Central Toronto,Davisville,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Pizza Place,Indian Restaurant,Middle Eastern Restaurant,Café,Dessert Shop,Gym
9,M4T,Central Toronto,"Moore Park, Summerhill East",0,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Grocery Store,Sandwich Place,Thai Restaurant,Park,Café,Restaurant
10,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Park,Sushi Restaurant,Gym,Thai Restaurant,Italian Restaurant,Gym / Fitness Center,Spa,Sandwich Place,Café


### Cluster 2 - Outlier

In [190]:
c2 = neighborhood_toronto_merged.loc[neighborhood_toronto_merged['Cluster Labels'] == 1].drop(['Latitude','Longitude'], axis=1)
print('The most common venues are {}'.format(c2['1st Most Common Venue'].unique()))
c2.head(30)

The most common venues are ['Café']


,Postcode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M4N,Central Toronto,Lawrence Park,1,Café,Coffee Shop,Pool,Bookstore,Gym / Fitness Center,College Quad,Park,College Gym,Trail,Ethiopian Restaurant


### Cluster 3 - Coffee shops and Hotels

In [191]:
c3 = neighborhood_toronto_merged.loc[neighborhood_toronto_merged['Cluster Labels'] == 2].drop(['Latitude','Longitude'], axis=1)
print('The most common venues are {}'.format(c3['1st Most Common Venue'].unique()))
c3.head(30)

The most common venues are ['Café' 'Coffee Shop' 'Hotel']


,Postcode,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M4X,Downtown Toronto,"Cabbagetown, St. James Town",2,Café,Restaurant,Coffee Shop,Gastropub,Japanese Restaurant,Park,Diner,Rock Club,Caribbean Restaurant,Jewelry Store
19,M5C,Downtown Toronto,St. James Town,2,Coffee Shop,Café,Restaurant,Bakery,Seafood Restaurant,Cosmetics Shop,Hotel,Breakfast Spot,Cocktail Bar,Diner
20,M5E,Downtown Toronto,Berczy Park,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Bakery,Cocktail Bar,Park,Farmers Market
22,M5H,Downtown Toronto,"Adelaide, King, Richmond",2,Café,Hotel,Coffee Shop,Theater,Restaurant,Sushi Restaurant,Steakhouse,Concert Hall,Bakery,Bar
23,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",2,Hotel,Coffee Shop,Restaurant,Café,Italian Restaurant,Park,Bar,Japanese Restaurant,Aquarium,Brewery
24,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",2,Café,Hotel,Coffee Shop,Steakhouse,Restaurant,Italian Restaurant,Concert Hall,Asian Restaurant,Park,Seafood Restaurant
25,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",2,Café,Hotel,Coffee Shop,Restaurant,Steakhouse,Concert Hall,Japanese Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Gastropub
26,M5S,Downtown Toronto,"Harbord, University of Toronto",2,Café,Vegetarian / Vegan Restaurant,Bakery,Bar,Coffee Shop,Restaurant,Mexican Restaurant,Bookstore,Ice Cream Shop,Comfort Food Restaurant
27,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",2,Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dessert Shop,Chinese Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Art Gallery
28,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",2,Coffee Shop,Café,Harbor / Marina,Dog Run,Sculpture Garden,Park,Track,Dance Studio,Garden,Scenic Lookout
